# Load Notebook

In [1]:
from scipy.special import boxcox1p
import dill
dill.load_session('../Notebook_Saves/preprocess.db')

# Additional Imports

In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /anaconda3/envs/ds/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0y/6r5660pn4tg8w4vhl1fl180h0000gn/T/tmph7xbzly5
  JVM stdout: /var/folders/0y/6r5660pn4tg8w4vhl1fl180h0000gn/T/tmph7xbzly5/h2o_megaman_started_from_python.out
  JVM stderr: /var/folders/0y/6r5660pn4tg8w4vhl1fl180h0000gn/T/tmph7xbzly5/h2o_megaman_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_megaman_lh8hqw
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# Dataframe Setup

Bring features and labels together again as a dataframe and make labels column to be categorical for proper binary classification when running H2O AutoML.

In [3]:
type_dict = {}
for col,typ in zip(house_train.dtypes.index, house_train.dtypes):
    typ = str(typ)
    if 'int' in typ:
        type_dict[col] = 'int'
    elif typ == 'category':
        type_dict[col] = 'categorical'
    elif 'float' in typ:
        type_dict[col] = 'float'
type_dict

{'Id': 'int',
 'MSSubClass': 'categorical',
 'LotFrontage': 'float',
 'LotArea': 'int',
 'OverallQual': 'categorical',
 'OverallCond': 'categorical',
 'YearBuilt': 'categorical',
 'YearRemodAdd': 'categorical',
 'MasVnrArea': 'float',
 'BsmtFinSF1': 'int',
 'BsmtFinSF2': 'int',
 'BsmtUnfSF': 'int',
 'TotalBsmtSF': 'int',
 '2ndFlrSF': 'int',
 'LowQualFinSF': 'int',
 'GrLivArea': 'int',
 'BsmtFullBath': 'int',
 'BsmtHalfBath': 'int',
 'FullBath': 'int',
 'HalfBath': 'int',
 'BedroomAbvGr': 'int',
 'KitchenAbvGr': 'int',
 'Fireplaces': 'int',
 'GarageYrBlt': 'categorical',
 'GarageCars': 'int',
 'WoodDeckSF': 'int',
 'OpenPorchSF': 'int',
 'EnclosedPorch': 'int',
 '3SsnPorch': 'int',
 'ScreenPorch': 'int',
 'PoolArea': 'int',
 'MiscVal': 'int',
 'MoSold': 'categorical',
 'YrSold': 'categorical',
 'SalePrice': 'float',
 'MSZoning': 'categorical',
 'Street': 'categorical',
 'Alley': 'categorical',
 'LotShape': 'categorical',
 'LandContour': 'categorical',
 'Utilities': 'categorical',
 'LotC

In [4]:
modeling_df = h2o.H2OFrame(house_train, column_names=house_train.columns.to_list(), column_types=type_dict)

Parse progress: |█████████████████████████████████████████████████████████| 100%


<br>
<br>
<br>

# Modeling

## Train/Test Split

In [5]:
# 80/20 split for training and validation
train, valid = modeling_df.split_frame(ratios=[.8], seed=28)

In [6]:
y = response
x = modeling_df.columns
x.remove(y)
x.remove('Id')

## AutoML

In [7]:
aml = H2OAutoML(balance_classes=True, max_models=10, seed=28, nfolds=10, stopping_metric='rmsle')

In [8]:
aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200112_132817,0.0150177,0.122547,0.0150177,0.0804122,0.00943299
StackedEnsemble_AllModels_AutoML_20200112_132817,0.0150551,0.122699,0.0150551,0.0802612,0.00944719
XGBoost_1_AutoML_20200112_132817,0.016513,0.128503,0.016513,0.0877849,0.00988617
GLM_1_AutoML_20200112_132817,0.0168204,0.129693,0.0168204,0.083976,0.00993249
XGBoost_2_AutoML_20200112_132817,0.016861,0.12985,0.016861,0.0869421,0.0100337
XGBoost_3_AutoML_20200112_132817,0.0176717,0.132935,0.0176717,0.0897701,0.0102226
GBM_5_AutoML_20200112_132817,0.0205999,0.143527,0.0205999,0.0995235,0.0111183
GBM_2_AutoML_20200112_132817,0.0210535,0.145098,0.0210535,0.102071,0.0112191
GBM_1_AutoML_20200112_132817,0.0221912,0.148967,0.0221912,0.103202,0.0115391
GBM_3_AutoML_20200112_132817,0.0223183,0.149393,0.0223183,0.103466,0.0115451


<br>
<br>
<br>

# Best Model

In [10]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20200112_132817

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.004867908507433064
RMSE: 0.06977039850418704
MAE: 0.04624494646777462
RMSLE: 0.0053761298961500885
R^2: 0.9650525040501985
Mean Residual Deviance: 0.004867908507433064
Null degrees of freedom: 1084
Residual degrees of freedom: 1080
Null deviance: 151.13187903810558
Residual deviance: 5.281680730564874
AIC: -2686.6270080644044

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 0.010241605514855864
RMSE: 0.10120081775784158
MAE: 0.07352735659478417
RMSLE: 0.007869226064085733
R^2: 0.9282888576416769
Mean Residual Deviance: 0.010241605514855864
Null degrees of freedom: 273
Residual degrees of freedom: 269
Null deviance: 39.29699172730049
Residual deviance: 2.8061999110705065
AIC: -465.69702977342405

ModelMetricsRegres

In [11]:
h2o.save_model(aml.leader, path = "../Models/")

'/Users/megaman/Documents/Coding/Kaggle/house_prices/Models/StackedEnsemble_BestOfFamily_AutoML_20200112_132817'

In [12]:
leader = aml.leader

## Testing Predictions

In [13]:
house_test.drop('Id', axis=1, inplace=True)

In [15]:
type_dict = {}
for col,typ in zip(house_test.dtypes.index, house_test.dtypes):
    typ = str(typ)
    if 'int' in typ:
        type_dict[col] = 'int'
    elif typ == 'category':
        type_dict[col] = 'categorical'
    elif 'float' in typ:
        type_dict[col] = 'float'
type_dict

{'MSSubClass': 'categorical',
 'LotFrontage': 'float',
 'LotArea': 'int',
 'OverallQual': 'categorical',
 'OverallCond': 'categorical',
 'YearBuilt': 'categorical',
 'YearRemodAdd': 'categorical',
 'MasVnrArea': 'float',
 'BsmtFinSF1': 'float',
 'BsmtFinSF2': 'float',
 'BsmtUnfSF': 'float',
 'TotalBsmtSF': 'float',
 '2ndFlrSF': 'int',
 'LowQualFinSF': 'int',
 'GrLivArea': 'int',
 'BsmtFullBath': 'float',
 'BsmtHalfBath': 'float',
 'FullBath': 'int',
 'HalfBath': 'int',
 'BedroomAbvGr': 'int',
 'KitchenAbvGr': 'int',
 'Fireplaces': 'int',
 'GarageYrBlt': 'categorical',
 'GarageCars': 'float',
 'WoodDeckSF': 'int',
 'OpenPorchSF': 'int',
 'EnclosedPorch': 'int',
 '3SsnPorch': 'int',
 'ScreenPorch': 'int',
 'PoolArea': 'int',
 'MiscVal': 'int',
 'MoSold': 'categorical',
 'YrSold': 'categorical',
 'MSZoning': 'categorical',
 'Street': 'categorical',
 'Alley': 'categorical',
 'LotShape': 'categorical',
 'LandContour': 'categorical',
 'Utilities': 'categorical',
 'LotConfig': 'categorical',


In [17]:
test_df = h2o.H2OFrame(house_test, column_names=house_test.columns.to_list(), column_types=type_dict)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
preds = leader.predict(test_df)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [27]:
submission.SalePrice = np.exp(leader.predict(test_df).as_data_frame())
submission.SalePrice

stackedensemble prediction progress: |████████████████████████████████████| 100%


/anaconda3/envs/ds/lib/python3.7/site-packages/h2o/job.py:70: UserWarning:

Test/Validation dataset column 'MSSubClass' has levels not trained on: [150]

/anaconda3/envs/ds/lib/python3.7/site-packages/h2o/job.py:70: UserWarning:

Test/Validation dataset column 'OverallQual' has levels not trained on: [1]

/anaconda3/envs/ds/lib/python3.7/site-packages/h2o/job.py:70: UserWarning:

Test/Validation dataset column 'OverallCond' has levels not trained on: [1]

/anaconda3/envs/ds/lib/python3.7/site-packages/h2o/job.py:70: UserWarning:

Test/Validation dataset column 'YearBuilt' has levels not trained on: [1879, 1895, 1896, 1901, 1902, 1907]

/anaconda3/envs/ds/lib/python3.7/site-packages/h2o/job.py:70: UserWarning:

Test/Validation dataset column 'GarageYrBlt' has levels not trained on: [1895.0, 1896.0, 1917.0, 1919.0, 1943.0, 2208.0]

/anaconda3/envs/ds/lib/python3.7/site-packages/h2o/job.py:70: UserWarning:

Test/Validation dataset column 'MSZoning' has levels not trained on: [NA]

/anacon

0       123320.074071
1       146883.194415
2       195136.670811
3       194127.773413
4       191912.235493
            ...      
1454     92540.434166
1455     82949.475898
1456    162928.502276
1457    129017.991092
1458    217194.325295
Name: SalePrice, Length: 1459, dtype: float64

In [28]:
submission.to_csv("../Submissions/submission_h2o_automl.csv", index=False)

Leaderboard Score: 

# Coefficients

In [29]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])

In [34]:
metalearner.coef()

{'Intercept': -0.43420629568560104,
 'XGBoost_1_AutoML_20200112_132817': 0.2843152883657562,
 'GLM_1_AutoML_20200112_132817': 0.3292694516453186,
 'XGBoost_2_AutoML_20200112_132817': 0.17549200191488645,
 'XGBoost_3_AutoML_20200112_132817': 0.09476231151554838,
 'GBM_5_AutoML_20200112_132817': 0.10958845282147013,
 'GBM_2_AutoML_20200112_132817': 0.04268511685563883,
 'GBM_1_AutoML_20200112_132817': 0.0,
 'GBM_3_AutoML_20200112_132817': 0.0,
 'GBM_4_AutoML_20200112_132817': 0.0,
 'DRF_1_AutoML_20200112_132817': 0.0}

<br>
<br>
<br>

# Save Notebook

In [38]:
import dill
dill.dump_session('../Notebook_Saves/h2o.db')

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  metalearner_AUTO_StackedEnsemble_AllModels_AutoML_20200112_132817


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gaussian,identity,"Elastic Net (alpha = 0.5, lambda = 2.711E-5 )","nlambda = 100, lambda.max = 0.247, lambda.min = 2.711E-5, lambda.1...",10,6,100,levelone_training_StackedEnsemble_AllModels_AutoML_20200112_132817




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.014758648184861353
RMSE: 0.12148517681125279
MAE: 0.08004495637212711
RMSLE: 0.009360770754608836
R^2: 0.8940452975076626
Mean Residual Deviance: 0.014758648184861353
Null degrees of freedom: 1084
Residual degrees of freedom: 1078
Null deviance: 151.13187903810558
Residual deviance: 16.013133280574568
AIC: -1479.1831476250943

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.010033284073284346
RMSE: 0.10016628211770838
MAE: 0.07187284773045305
RMSLE: 0.007786807401682876
R^2: 0.9297475125890049
Mean Residual Deviance: 0.010033284073284346
Null degrees of freedom: 273
Residual degrees of freedom: 267
Null deviance: 39.29699172730049
Residual deviance: 2.7491198360799105
AIC: -467.3278455236908

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 0.015055102761687693
RMSE: 0.12269923700532001
MAE: 0.08026118539976002
RMSLE: 0.009447194525611886
R^2: 0.8919170025516894

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,mae,0.080643065,0.013954141,0.091322035,0.103139356,0.07511949,0.0777699,0.071133316,0.073663816,0.07162261,0.07277543,0.06526761,0.10461713
1,mean_residual_deviance,0.014665896,0.007128044,0.0229698,0.029517999,0.010553965,0.015047344,0.009015106,0.009826974,0.011796966,0.009594085,0.008650565,0.019686157
2,mse,0.014665896,0.007128044,0.0229698,0.029517999,0.010553965,0.015047344,0.009015106,0.009826974,0.011796966,0.009594085,0.008650565,0.019686157
3,null_deviance,15.145896,2.9911292,20.788855,16.715466,17.051123,17.868708,15.806363,13.855181,12.034691,13.21635,11.497697,12.624528
4,r2,0.894749,0.044045422,0.87605506,0.79710627,0.9312109,0.9115691,0.9329394,0.9218798,0.9016475,0.91805273,0.9149266,0.8421024
5,residual_deviance,1.5941198,0.80837375,2.5726175,3.365052,1.1714902,1.5799712,1.0187069,1.0809672,1.1796967,1.026567,0.97751373,1.9686158
6,rmse,0.11827241,0.02743748,0.15155791,0.17180803,0.102732494,0.12266762,0.09494791,0.0991311,0.10861384,0.0979494,0.09300841,0.14030737
7,rmsle,0.009123775,0.0021196855,0.011974472,0.01285589,0.007908478,0.009786053,0.0071738404,0.0076351524,0.008283002,0.00759487,0.0070921043,0.010933889



Scoring History: 


...


Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
0,,2020-01-12 13:29:04,0.000 sec,1,.25E0,1,0.139292,0.143420,0.139373,0.007975
1,,2020-01-12 13:29:04,0.001 sec,2,.23E0,3,0.132431,0.136283,0.139220,0.007971
2,,2020-01-12 13:29:04,0.001 sec,3,.21E0,4,0.122487,0.125941,0.135553,0.008613
3,,2020-01-12 13:29:04,0.001 sec,4,.19E0,4,0.110984,0.113953,0.128814,0.009285
4,,2020-01-12 13:29:04,0.002 sec,5,.17E0,4,0.099153,0.101585,0.119062,0.008984
5,,2020-01-12 13:29:04,0.002 sec,6,.16E0,4,0.087976,0.089865,0.109317,0.009183
6,,2020-01-12 13:29:04,0.002 sec,7,.14E0,5,0.077803,0.079162,0.097170,0.008489
7,,2020-01-12 13:29:04,0.003 sec,8,.13E0,5,0.068602,0.069445,0.086783,0.008474
8,,2020-01-12 13:29:04,0.003 sec,9,.12E0,5,0.060501,0.060854,0.077786,0.008751
9,,2020-01-12 13:29:04,0.003 sec,10,.11E0,5,0.053499,0.053398,0.070037,0.009127



See the whole table with table.as_data_frame()

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.004867908507433064
RMSE: 0.06977039850418704
MAE: 0.04624494646777462
RMSLE: 0.0053761298961500885
R^2: 0.9650525040501985
Mean Residual Deviance: 0.004867908507433064
Null degrees of freedom: 1084
Residual degrees of freedom: 1080
Null deviance: 151.13187903810558
Residual deviance: 5.281680730564874
AIC: -2686.6270080644044

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 0.010241605514855864
RMSE: 0.10120081775784158
MAE: 0.07352735659478417
RMSLE: 0.007869226064085733
R^2: 0.9282888576416769
Mean Residual Deviance: 0.010241605514855864
Null degrees of freedom: 273
Residual degrees of freedom: 269
Null deviance: 39.29699172730049
Residual deviance: 2.8061999110705065
AIC: -465.69702977342405

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.015017719544838744
RMSE: 0.1225468055

predict
11.7225
11.8974
12.1815
12.1763
12.1648
12.0557
12.0671
12.006
12.0754
11.7178


TypeError: no default __reduce__ due to non-trivial __cinit__